In [ ]:
from google.colab import userdata
openai_key = userdata.get('OPENAI')

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = openai_key

In [ ]:
from IPython.display import Image, display

In [ ]:
# que = "what is capital of india"
context = "capital of india is new delhi and"," economic capital is mumbai"

answer = "capital of india is new delhi"

-  create synthetic questions on give answer
- calculate mean cosine similariy of those question with original question

In [ ]:
!pip install langchain openai pypdf tiktoken chromadb

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/nvidia.pdf")
docs = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = splitter.split_documents(docs)

from langchain.embeddings.openai import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings()

from langchain.vectorstores import Chroma
db = Chroma.from_documents(chunks, embedding_model, persist_directory="db_")

retrival = db.as_retriever()
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI()

from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retrival)


### Important terms

- question is the user query.
- answer is the llm response. 
- context is documents retrieved from the vector store/ database. 
- ground_truth is the expected answer to user query.

In [ ]:
questions = ["how much money nvidia has invested in research?",
             "what is expiration dates for patent?",
             "what solution does nvidia offer for llms?"

]

ground_truths = [["Nvidia have invested over $45.3 billion in research and development"],
                 ["currently issued patents have expiration dates from February 2024 to August 2043"],
                 ["NeMo – a complete solution for building enterprise-ready Large Language Models"]]

In [ ]:
answers = []
contexts = []

In [ ]:
for que in questions:
  answers.append(qa_chain.invoke(que)['result'])
  contexts.append([doc.page_content for doc in retrival.get_relevant_documents(que)])

In [ ]:
eval_data = {
    "question" : questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : ground_truths
}

In [ ]:
# !pip install datasets

In [ ]:
from datasets import Dataset

data = Dataset.from_dict(eval_data)

In [ ]:
data

In [ ]:
import pandas as pd

df = pd.DataFrame(eval_data)

In [ ]:
df.head(3)

###Evaluating RAG application using ragas

In [ ]:
# !pip install ragas

In [ ]:
from ragas import  evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall

In [ ]:
score = evaluate(
    dataset = data,
    metrics = [faithfulness, answer_relevancy, context_precision, context_recall]
)

In [ ]:
score